In [432]:
import pandas as pd
import numpy as np
from rapidfuzz import process, fuzz
from rapidfuzz.string_metric import levenshtein, normalized_levenshtein

First step, find the top 50% debt holders in NH.

In [433]:
CD_all_post = pd.read_csv('../../../Data/Post1790/aggregated_CD_noname.csv', index_col=0).reset_index(drop=True)
CD_NH = CD_all_post.loc[CD_all_post.state=='NH']
CD_NH = CD_NH.copy()

In [434]:
CD_NH['total'] = CD_NH['6p_Dollar'] + CD_NH['6p_def_Dollar'] + CD_NH['3p_Dollar']
CD_NH = CD_NH.sort_values(by='total', ascending=False).dropna(subset='total').iloc[1:]
CD_NH['total_cum'] = CD_NH['total'].cumsum()
CD_NH_top_holders = CD_NH.loc[CD_NH.total_cum<=0.5*sum(CD_NH.total), :].reset_index()
CD_NH_top_holders

,index,6p_Dollar,6p_Cents,6p_def_Dollar,6p_def_Cents,3p_Dollar,3p_Cents,town,state,occupation,county,name_type,6p_total,total,total_cum
0,1790,12396.0,79.0,6198.0,39.0,7977.0,60.0,Portsmouth,NH,Esquire,Rockingham County,town,12396.79,26571.0,26571.0
1,1851,7719.0,65.0,3859.0,82.0,2958.0,49.0,Exeter,NH,Merchant,Rockingham County,town,7719.65,14536.0,41107.0
2,1805,5200.0,NaN,2600.0,NaN,2953.0,57.0,Newbury,NH,Esquire,Hillsborough County,town,NaN,10753.0,51860.0
3,1957,3695.0,83.0,1847.0,91.0,4992.0,7.0,Exeter,NH,Merchant,Rockingham County,town,3695.83,10534.0,62394.0
4,1787,5124.0,59.0,2562.0,29.0,1480.0,19.0,Portsmouth,NH,Widow,Rockingham County,town,5124.59,9166.0,71560.0
5,1780,4082.0,6.0,2041.0,3.0,2538.0,45.0,Merrimac,NH,Esquire,Hillsborough County,town,4082.06,8661.0,80221.0
6,1882,3975.0,85.0,1987.0,92.0,2296.0,4.0,Nottingham,NH,Esquire,Rockingham County,town,3975.85,8258.0,88479.0
7,1920,3270.0,59.0,1635.0,29.0,2818.0,83.0,Salem,NH,Merchant,Rockingham County,town,3270.59,7723.0,96202.0
8,1800,3606.0,29.0,1803.0,15.0,1503.0,8.0,Hampton,NH,Esquire,Rockingham County,town,3606.29,6912.0,103114.0
9,1799,3768.0,67.0,1884.0,34.0,1015.0,97.0,Kensington,NH,Esquire,Rockingham County,town,3768.67,6667.0,109781.0


In [435]:
top_holders = {26571.0: 'John Langdon', 
               14536.0: False, 
               10753.0: 'Edmund Sawyer',
               10534.0: False,
               9166.0: 'Susanna Atkinson',
               8661.0: 'Matthew Thornton',
               8258.0: 'Joseph Gilley',
               7723.0: 'Elias Hasket Derby',
               6912.0: 'Christopher Toppan',
               6667.0: 'Jeremiah Fogg',
               6515.0: 'Joseph Mills',
               6461.0: 'John Taylor Gilman',
               5917.0: 'Joshua Brackett',
               5464.0: 'John Lane'}

CD_NH_top_holders['name'] = CD_NH_top_holders.apply(lambda row: top_holders[row.total], axis=1)
CD_NH_top_holders = CD_NH_top_holders.query("name != False").reset_index(drop=True)

# split names
CD_NH_top_holders['first_name'], CD_NH_top_holders['last_name'] = (
    CD_NH_top_holders['name'].str.split(' ').str[0], CD_NH_top_holders['name'].str.split(' ').str[1]
)
CD_NH_top_holders

,index,6p_Dollar,6p_Cents,6p_def_Dollar,6p_def_Cents,3p_Dollar,3p_Cents,town,state,occupation,county,name_type,6p_total,total,total_cum,name,first_name,last_name
0,1790,12396.0,79.0,6198.0,39.0,7977.0,60.0,Portsmouth,NH,Esquire,Rockingham County,town,12396.79,26571.0,26571.0,John Langdon,John,Langdon
1,1805,5200.0,NaN,2600.0,NaN,2953.0,57.0,Newbury,NH,Esquire,Hillsborough County,town,NaN,10753.0,51860.0,Edmund Sawyer,Edmund,Sawyer
2,1787,5124.0,59.0,2562.0,29.0,1480.0,19.0,Portsmouth,NH,Widow,Rockingham County,town,5124.59,9166.0,71560.0,Susanna Atkinson,Susanna,Atkinson
3,1780,4082.0,6.0,2041.0,3.0,2538.0,45.0,Merrimac,NH,Esquire,Hillsborough County,town,4082.06,8661.0,80221.0,Matthew Thornton,Matthew,Thornton
4,1882,3975.0,85.0,1987.0,92.0,2296.0,4.0,Nottingham,NH,Esquire,Rockingham County,town,3975.85,8258.0,88479.0,Joseph Gilley,Joseph,Gilley
5,1920,3270.0,59.0,1635.0,29.0,2818.0,83.0,Salem,NH,Merchant,Rockingham County,town,3270.59,7723.0,96202.0,Elias Hasket Derby,Elias,Hasket
6,1800,3606.0,29.0,1803.0,15.0,1503.0,8.0,Hampton,NH,Esquire,Rockingham County,town,3606.29,6912.0,103114.0,Christopher Toppan,Christopher,Toppan
7,1799,3768.0,67.0,1884.0,34.0,1015.0,97.0,Kensington,NH,Esquire,Rockingham County,town,3768.67,6667.0,109781.0,Jeremiah Fogg,Jeremiah,Fogg
8,1793,3677.0,87.0,1838.0,94.0,1000.0,30.0,Deerfield,NH,Esquire,Rockingham County,town,3677.87,6515.0,116296.0,Joseph Mills,Joseph,Mills
9,1794,2482.0,28.0,1241.0,14.0,2738.0,61.0,Exeter,NH,Esquire,Rockingham County,town,2482.28,6461.0,122757.0,John Taylor Gilman,John,Taylor


Now move on to match names with pre-1790 records.

In [436]:
loan = pd.read_csv('../../../Data/Pre1790/cleaned/loan_office_certificates_9_states_cleaned.csv', index_col = 0)
marine = pd.read_csv('../../../Data/Pre1790/cleaned/Marine_Liquidated_Debt_Certificates_cleaned.csv', index_col = 0)
pierce = pd.read_csv('../../../Data/Pre1790/cleaned/Pierce_Certs_cleaned_2021.csv', index_col = 0)

# liquidCT = pd.read_csv('../../../Data/Pre1790/cleaned/liquidated_debt_certificates_CT_cleaned.csv', index_col = 0)
# liquidDE = pd.read_csv('../../../Data/Pre1790/cleaned/liquidated_debt_certificates_DE_cleaned.csv', index_col = 0)
# liquidMA = pd.read_csv('../../../Data/Pre1790/cleaned/liquidated_debt_certificates_MA_cleaned.csv', index_col = 0)
# liquidNJ = pd.read_csv('../../../Data/Pre1790/cleaned/liquidated_debt_certificates_NJ_cleaned.csv', index_col = 0)
# liquidNH = pd.read_csv('../../../Data/Pre1790/cleaned/liquidated_debt_certificates_NH_cleaned.csv', index_col = 0)
# liquidNY = pd.read_csv('../../../Data/Pre1790/cleaned/liquidated_debt_certificates_NY_cleaned.csv', index_col = 0)
# liquidPA1 = pd.read_csv('../../../Data/Pre1790/cleaned/liquidated_debt_certificates_PA_story_cleaned.csv', index_col = 0)
# liquidRI = pd.read_csv('../../../Data/Pre1790/cleaned/liquidated_debt_certificates_RI_cleaned.csv', index_col = 0)
# liquidPA2 = pd.read_csv('../../../Data/Pre1790/cleaned/liquidated_debt_certificates_PA_stelle_cleaned.csv', index_col = 0)

pierce = pierce.rename({'First': 'First name', 'Last': 'Last name'}, axis=1)
loan = loan.rename({'First Name 1 ': 'First name', 'Last Name 1 ': 'Last name'}, axis=1)
pierce['exact'] = 1
loan['exact'] = 1
marine['exact'] = 1

# dict of directory
state_list = ['CT', 'DE', 'MA', 'NJ', 'NH', 'NY', 'RI', 'PA_stelle', 'PA_story', ]
dirs = [
    (state, f'../../../Data/Pre1790/cleaned/liquidated_debt_certificates_{state}_cleaned.csv') for state in state_list
]
dirs = dict(dirs)

# concat
df_liquid = pd.DataFrame({})
for state in state_list:

    df_cur = pd.read_csv(dirs[state], index_col = 0)
    # new column
    df_cur.insert(loc=0, column='state', value=state)
    df_liquid = pd.concat([df_liquid, df_cur], axis=0)
df_liquid['exact'] = 1

df_liquid

/var/folders/bs/pwc2w2k54fl8yr3kswpvkkw80000gn/T/ipykernel_55756/2718079459.py:3: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  pierce = pd.read_csv('../../../Data/Pre1790/cleaned/Pierce_Certs_cleaned_2021.csv', index_col = 0)


,state,Register Page,JPEG number,Number,Letter,Date of the Certificate: Month,Date of the Certificate: Day,Date of the Certificate: Year,First name,Last name,...,First name 3,Last name 3,8th,Date of the Certificate: Date,Line Strike Thorugh?,10th,Exchange,Amount in Specie_1,Amount in Specie_2,exact
0,CT,4.0,256.0,1.0,A,6.0,28.0,1783.0,Thomas,Dickenson,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,CT,4.0,256.0,2.0,B,6.0,30.0,1783.0,Paul,Noyes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,CT,4.0,256.0,3.0,C,7.0,1.0,1783.0,William,Redfield,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,CT,4.0,256.0,4.0,D,7.0,2.0,1783.0,Eleazer,Conant,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,CT,4.0,256.0,5.0,E,7.0,3.0,1783.0,Prentice,Hoomer,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
950,PA_story,NaN,449.0,951.0,R,8.0,NaN,1786.0,Shadrick,Hyatt,...,NaN,NaN,NaN,29.0,Lo 50433,NaN,NaN,66.0,NaN,1
951,PA_story,NaN,449.0,952.0,D,8.0,NaN,1786.0,Nathaniel,Chirckfield,...,NaN,NaN,NaN,29.0,Lo 50433,NaN,NaN,66.0,NaN,1
952,PA_story,NaN,449.0,953.0,E,8.0,NaN,1786.0,Christopher,Armstrong,...,NaN,NaN,NaN,29.0,2191,NaN,NaN,11.0,60.0,1
953,PA_story,NaN,449.0,954.0,G,8.0,NaN,1786.0,Ebenezer,Myers,...,NaN,NaN,NaN,29.0,1850,NaN,NaN,32.0,17.0,1


In [437]:
def match_names(row, first_name, last_name):
    # sanity
    if (type(row['First name']) != str) or type(row['Last name']) != str:
        return False
    
    # choose levenshtein and allow for one mistyped character
    first_score = process.cdist([row['First name']], [first_name], scorer=levenshtein)[0][0]
    last_score = process.cdist([row['Last name']], [last_name], scorer=levenshtein)[0][0]
    if (first_score <= 1) & (last_score <= 1):
        return True
    else:
        return False

# allow for multiple first_names
def get_result1(df, first_names, last_name):
    df_out = pd.DataFrame({})

    for f_name in first_names:
        df_cur = df.loc[df.apply(lambda row: match_names(row, f_name, last_name), axis=1)]
        df_cur.loc[(df_cur['First name']!=f_name) | (df_cur['Last name']!=last_name), 'exact'] = 0
        df_out = pd.concat([df_out, df_cur], axis=0)
    # fetch all possible cases
    return df_out

### 'John Langdon'

In [438]:
f_names, l_name = ['John'], 'Langdon'

loan1_JLD = get_result1(loan, f_names, l_name)
marine1_JLD = get_result1(marine, f_names, l_name)
pierce1_JLD = get_result1(pierce, f_names, l_name)
liquid1_JLD = get_result1(df_liquid, f_names, l_name)

In [439]:
loan1_JLD

,State,Year,Month,Day,Title 1,First name,Last name,Title 2,First Name 2,Last Name 2,Title 3,First Name 3,Last Name 3,Face Value,Specie Value,notes,original text,match eligible,exact
133,1,1778,5,7.0,Hon,John,Langdon,NaN,NaN,NaN,NaN,NaN,NaN,200,84.57778,NaN,NaN,NaN,1
134,1,1778,5,7.0,Hon,John,Langdon,NaN,NaN,NaN,NaN,NaN,NaN,200,84.57778,NaN,NaN,NaN,1
135,1,1778,5,7.0,Hon,John,Langdon,NaN,NaN,NaN,NaN,NaN,NaN,200,84.57778,NaN,NaN,NaN,1
136,1,1778,5,7.0,Hon,John,Langdon,NaN,NaN,NaN,NaN,NaN,NaN,200,84.57778,NaN,NaN,NaN,1
137,1,1778,5,7.0,Hon,John,Langdon,NaN,NaN,NaN,NaN,NaN,NaN,200,84.57778,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1668,1,1778,5,22.0,Esq,John,Langdon,NaN,NaN,NaN,NaN,NaN,NaN,600,237.18890,NaN,NaN,NaN,1
1669,1,1778,5,22.0,Esq,John,Langdon,NaN,NaN,NaN,NaN,NaN,NaN,600,237.18890,NaN,NaN,NaN,1
1670,1,1778,6,4.0,Esq,John,Langdon,NaN,NaN,NaN,NaN,NaN,NaN,600,223.73330,NaN,NaN,NaN,1
1671,1,1778,6,4.0,Esq,John,Langdon,NaN,NaN,NaN,NaN,NaN,NaN,600,223.73330,NaN,NaN,NaN,1


In [440]:
marine1_JLD

,Page,JPEG number,Number,Letter,Date of the Certificate: Month,Date of the Certificate: Day,Date of the Certificate: Year,First name,Last name,Title,...,Total Dollars_1,Total Dollars_2,Line Strike Thorugh: Yes?,Line Strike Thorugh: Note,Notes,original text,cleaning notes,first name 2,last name 2,exact


In [441]:
pierce1_JLD

,CN,Last name,First name,Value,Group,To Whom Issued,State,Officer,Last 2,First 2,original text,exact


In [442]:
liquid1_JLD = liquid1_JLD.query("state=='NH'")

In [443]:
print(
    f"loan: {loan1_JLD['Face Value'].sum()} \n" + 
    f"marine: {marine1_JLD['Dollars'].sum()} \n" + 
    f"pierce: {pierce1_JLD['Value'].sum()} \n" + 
    f"liquid: {liquid1_JLD['Dollars'].sum()}"
)

loan: 42200 
marine: 0.0 
pierce: 0.0 
liquid: 48.0


### 'Edmund Sawyer'

In [444]:
f_names, l_name = ['Edmund'], 'Sawyer'

loan1_ES = get_result1(loan, f_names, l_name)
marine1_ES = get_result1(marine, f_names, l_name)
pierce1_ES = get_result1(pierce, f_names, l_name)
liquid1_ES = get_result1(df_liquid, f_names, l_name)

In [445]:
loan1_ES = loan1_ES.query("State==1")

In [446]:
marine1_ES

,Page,JPEG number,Number,Letter,Date of the Certificate: Month,Date of the Certificate: Day,Date of the Certificate: Year,First name,Last name,Title,...,Total Dollars_1,Total Dollars_2,Line Strike Thorugh: Yes?,Line Strike Thorugh: Note,Notes,original text,cleaning notes,first name 2,last name 2,exact


In [447]:
pierce1_ES

,CN,Last name,First name,Value,Group,To Whom Issued,State,Officer,Last 2,First 2,original text,exact


In [448]:
liquid1_ES = liquid1_ES.query("state!='MA'")

In [449]:
print(
    f"loan: {loan1_ES['Face Value'].sum()} \n" + 
    f"marine: {marine1_ES['Dollars'].sum()} \n" + 
    f"pierce: {pierce1_ES['Value'].sum()} \n" + 
    f"liquid: {liquid1_ES['Dollars'].sum()}"
)

loan: 4900 
marine: 0.0 
pierce: 0.0 
liquid: 0.0


### 'Susanna Atkinson'

In [450]:
f_names, l_name = ['Susanna', 'George King', 'George'], 'Atkinson'
# George King Atkinson is her spouse

loan1_SA = get_result1(loan, f_names, l_name)
marine1_SA = get_result1(marine, f_names, l_name)
pierce1_SA = get_result1(pierce, f_names, l_name)
liquid1_SA = get_result1(df_liquid, f_names, l_name)

In [451]:
loan1_SA

,State,Year,Month,Day,Title 1,First name,Last name,Title 2,First Name 2,Last Name 2,Title 3,First Name 3,Last Name 3,Face Value,Specie Value,notes,original text,match eligible,exact


In [452]:
marine1_SA

,Page,JPEG number,Number,Letter,Date of the Certificate: Month,Date of the Certificate: Day,Date of the Certificate: Year,First name,Last name,Title,...,Total Dollars_1,Total Dollars_2,Line Strike Thorugh: Yes?,Line Strike Thorugh: Note,Notes,original text,cleaning notes,first name 2,last name 2,exact


In [453]:
pierce1_SA

,CN,Last name,First name,Value,Group,To Whom Issued,State,Officer,Last 2,First 2,original text,exact


In [454]:
liquid1_SA

,state,Register Page,JPEG number,Number,Letter,Date of the Certificate: Month,Date of the Certificate: Day,Date of the Certificate: Year,First name,Last name,...,First name 3,Last name 3,8th,Date of the Certificate: Date,Line Strike Thorugh?,10th,Exchange,Amount in Specie_1,Amount in Specie_2,exact
6271,PA_stelle,NaN,372.0,6272.0,A,10.0,NaN,1785.0,George,Atkinson,...,NaN,NaN,NaN,8.0,1850,NaN,NaN,NaN,NaN,1
7,PA_story,NaN,434.0,8.0,J,6.0,NaN,1785.0,George,Atkinson,...,NaN,NaN,NaN,29.0,2345,NaN,61.5,38.0,12.0,1


In [455]:
print(
    f"loan: {loan1_SA['Face Value'].sum()} \n" + 
    f"marine: {marine1_SA['Dollars'].sum()} \n" + 
    f"pierce: {pierce1_SA['Value'].sum()} \n" + 
    f"liquid: {liquid1_SA['Dollars'].sum()}"
)

loan: 0 
marine: 0.0 
pierce: 0.0 
liquid: 2346.0


### 'Matthew Thornton'

In [456]:
f_names, l_name = ['Matthew'], 'Thornton'

loan1_MT = get_result1(loan, f_names, l_name)
marine1_MT = get_result1(marine, f_names, l_name)
pierce1_MT = get_result1(pierce, f_names, l_name)
liquid1_MT = get_result1(df_liquid, f_names, l_name)

In [457]:
loan1_MT

,State,Year,Month,Day,Title 1,First name,Last name,Title 2,First Name 2,Last Name 2,Title 3,First Name 3,Last Name 3,Face Value,Specie Value,notes,original text,match eligible,exact
30,1,1777,10,30.0,Hon,Matthew,Thornton,NaN,NaN,NaN,NaN,NaN,NaN,200,166.64440,NaN,NaN,NaN,1
626,1,1777,7,8.0,Hon,Matthew,Thornton,NaN,NaN,NaN,NaN,NaN,NaN,300,300.00000,NaN,NaN,NaN,1
627,1,1777,7,8.0,Hon,Matthew,Thornton,NaN,NaN,NaN,NaN,NaN,NaN,300,300.00000,NaN,NaN,NaN,1
662,1,1777,8,29.0,Hon,Matthew,Thornton,NaN,NaN,NaN,NaN,NaN,NaN,300,300.00000,NaN,NaN,NaN,1
787,1,1778,7,30.0,Honble,Matthew,Thornton,NaN,NaN,NaN,NaN,NaN,NaN,300,86.97778,NaN,NaN,NaN,1
789,1,1778,8,25.0,Honble,Matthew,Thornton,NaN,NaN,NaN,NaN,NaN,NaN,300,77.38889,NaN,NaN,NaN,1
1108,1,1777,7,8.0,Hon,Matthew,Thornton,NaN,NaN,NaN,NaN,NaN,NaN,400,400.00000,NaN,NaN,NaN,1
1149,1,1777,11,27.0,Hon,Matthew,Thornton,NaN,NaN,NaN,NaN,NaN,NaN,400,305.64440,NaN,NaN,NaN,1
1155,1,1778,1,21.0,Hon,Matthew,Thornton,NaN,NaN,NaN,NaN,NaN,NaN,400,257.85560,NaN,NaN,NaN,1
1159,1,1778,3,14.0,Hon,Matthew,Thornton,NaN,NaN,NaN,NaN,NaN,NaN,400,215.60000,NaN,NaN,NaN,1


In [458]:
marine1_MT

,Page,JPEG number,Number,Letter,Date of the Certificate: Month,Date of the Certificate: Day,Date of the Certificate: Year,First name,Last name,Title,...,Total Dollars_1,Total Dollars_2,Line Strike Thorugh: Yes?,Line Strike Thorugh: Note,Notes,original text,cleaning notes,first name 2,last name 2,exact


In [459]:
pierce1_MT

,CN,Last name,First name,Value,Group,To Whom Issued,State,Officer,Last 2,First 2,original text,exact


In [460]:
liquid1_MT

,state,Register Page,JPEG number,Number,Letter,Date of the Certificate: Month,Date of the Certificate: Day,Date of the Certificate: Year,First name,Last name,...,First name 3,Last name 3,8th,Date of the Certificate: Date,Line Strike Thorugh?,10th,Exchange,Amount in Specie_1,Amount in Specie_2,exact


In [461]:
print(
    f"loan: {loan1_MT['Face Value'].sum()} \n" + 
    f"marine: {marine1_MT['Dollars'].sum()} \n" + 
    f"pierce: {pierce1_MT['Value'].sum()} \n" + 
    f"liquid: {liquid1_MT['Dollars'].sum()}"
)

loan: 14800 
marine: 0.0 
pierce: 0.0 
liquid: 0.0


### 'Joseph Gilley'

In [462]:
f_names, l_name = ['Joseph'], 'Gilley'

loan1_JG = get_result1(loan, f_names, l_name)
marine1_JG = get_result1(marine, f_names, l_name)
pierce1_JG = get_result1(pierce, f_names, l_name)
liquid1_JG = get_result1(df_liquid, f_names, l_name)

In [463]:
loan1_JG

,State,Year,Month,Day,Title 1,First name,Last name,Title 2,First Name 2,Last Name 2,Title 3,First Name 3,Last Name 3,Face Value,Specie Value,notes,original text,match eligible,exact


In [464]:
marine1_JG

,Page,JPEG number,Number,Letter,Date of the Certificate: Month,Date of the Certificate: Day,Date of the Certificate: Year,First name,Last name,Title,...,Total Dollars_1,Total Dollars_2,Line Strike Thorugh: Yes?,Line Strike Thorugh: Note,Notes,original text,cleaning notes,first name 2,last name 2,exact


In [465]:
pierce1_JG = pierce1_JG.query("exact==1")

In [466]:
liquid1_JG

,state,Register Page,JPEG number,Number,Letter,Date of the Certificate: Month,Date of the Certificate: Day,Date of the Certificate: Year,First name,Last name,...,First name 3,Last name 3,8th,Date of the Certificate: Date,Line Strike Thorugh?,10th,Exchange,Amount in Specie_1,Amount in Specie_2,exact
116,NH,NaN,94.0,117.0,D,6.0,8.0,1784.0,Joseph,Cilley,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [467]:
print(
    f"loan: {loan1_JG['Face Value'].sum()} \n" + 
    f"marine: {marine1_JG['Dollars'].sum()} \n" + 
    f"pierce: {pierce1_JG['Value'].sum()} \n" + 
    f"liquid: {liquid1_JG['Dollars'].sum()}"
)

loan: 0 
marine: 0.0 
pierce: 0.0 
liquid: 1001.0


### 'Elias Hasket Derby'

In [468]:
f_names, l_name = ['Elias', 'Elias Hasket', 'Hasket'], 'Derby'

loan1_ED = get_result1(loan, f_names, l_name)
marine1_ED = get_result1(marine, f_names, l_name)
pierce1_ED = get_result1(pierce, f_names, l_name)
liquid1_ED = get_result1(df_liquid, f_names, l_name)

In [469]:
loan1_ED

,State,Year,Month,Day,Title 1,First name,Last name,Title 2,First Name 2,Last Name 2,Title 3,First Name 3,Last Name 3,Face Value,Specie Value,notes,original text,match eligible,exact


In [470]:
marine1_ED

,Page,JPEG number,Number,Letter,Date of the Certificate: Month,Date of the Certificate: Day,Date of the Certificate: Year,First name,Last name,Title,...,Total Dollars_1,Total Dollars_2,Line Strike Thorugh: Yes?,Line Strike Thorugh: Note,Notes,original text,cleaning notes,first name 2,last name 2,exact


In [471]:
pierce1_ED

,CN,Last name,First name,Value,Group,To Whom Issued,State,Officer,Last 2,First 2,original text,exact


In [472]:
liquid1_ED

,state,Register Page,JPEG number,Number,Letter,Date of the Certificate: Month,Date of the Certificate: Day,Date of the Certificate: Year,First name,Last name,...,First name 3,Last name 3,8th,Date of the Certificate: Date,Line Strike Thorugh?,10th,Exchange,Amount in Specie_1,Amount in Specie_2,exact


In [473]:
print(
    f"loan: {loan1_ED['Face Value'].sum()} \n" + 
    f"marine: {marine1_ED['Dollars'].sum()} \n" + 
    f"pierce: {pierce1_ED['Value'].sum()} \n" + 
    f"liquid: {liquid1_ED['Dollars'].sum()}"
)

loan: 0 
marine: 0.0 
pierce: 0.0 
liquid: 0.0


### 'Christopher Toppan'

In [474]:
f_names, l_name = ['Christopher'], 'Toppan'

loan1_CT = get_result1(loan, f_names, l_name)
marine1_CT = get_result1(marine, f_names, l_name)
pierce1_CT = get_result1(pierce, f_names, l_name)
liquid1_CT = get_result1(df_liquid, f_names, l_name)

In [475]:
loan1_CT 

,State,Year,Month,Day,Title 1,First name,Last name,Title 2,First Name 2,Last Name 2,Title 3,First Name 3,Last Name 3,Face Value,Specie Value,notes,original text,match eligible,exact
5513,2,1777,11,4.0,NaN,Christopher,Joppan,NaN,NaN,NaN,NaN,NaN,NaN,200,164.0917,NaN,NaN,NaN,0
5514,2,1777,11,4.0,NaN,Christopher,Joppan,NaN,NaN,NaN,NaN,NaN,NaN,200,164.0917,NaN,NaN,NaN,0
5515,2,1777,11,4.0,NaN,Christopher,Joppan,NaN,NaN,NaN,NaN,NaN,NaN,200,164.0917,NaN,NaN,NaN,0
5516,2,1777,11,4.0,NaN,Christopher,Joppan,NaN,NaN,NaN,NaN,NaN,NaN,200,164.0917,NaN,NaN,NaN,0
5517,2,1777,11,4.0,NaN,Christopher,Joppan,NaN,NaN,NaN,NaN,NaN,NaN,200,164.0917,NaN,NaN,NaN,0
5518,2,1777,11,4.0,NaN,Christopher,Joppan,NaN,NaN,NaN,NaN,NaN,NaN,200,164.0917,NaN,NaN,NaN,0


In [476]:
marine1_CT

,Page,JPEG number,Number,Letter,Date of the Certificate: Month,Date of the Certificate: Day,Date of the Certificate: Year,First name,Last name,Title,...,Total Dollars_1,Total Dollars_2,Line Strike Thorugh: Yes?,Line Strike Thorugh: Note,Notes,original text,cleaning notes,first name 2,last name 2,exact


In [477]:
pierce1_CT

,CN,Last name,First name,Value,Group,To Whom Issued,State,Officer,Last 2,First 2,original text,exact


In [478]:
liquid1_CT

,state,Register Page,JPEG number,Number,Letter,Date of the Certificate: Month,Date of the Certificate: Day,Date of the Certificate: Year,First name,Last name,...,First name 3,Last name 3,8th,Date of the Certificate: Date,Line Strike Thorugh?,10th,Exchange,Amount in Specie_1,Amount in Specie_2,exact


In [479]:
print(
    f"loan: {loan1_CT['Face Value'].sum()} \n" + 
    f"marine: {marine1_CT['Dollars'].sum()} \n" + 
    f"pierce: {pierce1_CT['Value'].sum()} \n" + 
    f"liquid: {liquid1_CT['Dollars'].sum()}"
)

loan: 1200 
marine: 0.0 
pierce: 0.0 
liquid: 0.0


### 'Jeremiah Fogg'

In [480]:
f_names, l_name = ['Jeremiah'], 'Fogg'

loan1_JF = get_result1(loan, f_names, l_name)
marine1_JF = get_result1(marine, f_names, l_name)
pierce1_JF = get_result1(pierce, f_names, l_name)
liquid1_JF = get_result1(df_liquid, f_names, l_name)

In [481]:
loan1_JF

,State,Year,Month,Day,Title 1,First name,Last name,Title 2,First Name 2,Last Name 2,Title 3,First Name 3,Last Name 3,Face Value,Specie Value,notes,original text,match eligible,exact
609,1,1777,5,23.0,NaN,Jeremiah,Fogg,NaN,NaN,NaN,NaN,NaN,NaN,300,300.00000,NaN,NaN,NaN,1
610,1,1777,5,26.0,Jun,Jeremiah,Fogg,NaN,NaN,NaN,NaN,NaN,NaN,300,300.00000,NaN,NaN,NaN,1
611,1,1777,5,26.0,Jun,Jeremiah,Fogg,NaN,NaN,NaN,NaN,NaN,NaN,300,300.00000,NaN,NaN,NaN,1
1073,1,1777,5,23.0,NaN,Jeremiah,Fogg,NaN,NaN,NaN,NaN,NaN,NaN,400,400.00000,NaN,NaN,NaN,1
1740,1,1779,3,26.0,Jun,Jeremiah,Fogg,NaN,NaN,NaN,NaN,NaN,NaN,600,55.38889,NaN,NaN,NaN,1


In [482]:
marine1_JF

,Page,JPEG number,Number,Letter,Date of the Certificate: Month,Date of the Certificate: Day,Date of the Certificate: Year,First name,Last name,Title,...,Total Dollars_1,Total Dollars_2,Line Strike Thorugh: Yes?,Line Strike Thorugh: Note,Notes,original text,cleaning notes,first name 2,last name 2,exact


In [483]:
pierce1_JF

,CN,Last name,First name,Value,Group,To Whom Issued,State,Officer,Last 2,First 2,original text,exact
28628,19434.0,Fogg,Jeremiah,400.00,33.0,Regiment under Lieut. Col. G. Reid; A. Morrell...,NH,0.0,NaN,NaN,NaN,1
28629,20489.0,Fogg,Jeremiah,50.40,35.0,Regiment under Lieut. Col. G. Reid New Hampsh...,NH,0.0,NaN,NaN,NaN,1
28630,21769.0,Fogg,Jeremiah,20.30,36.0,Regiment under Lieut. Col. G. Reid New Hampsh...,NH,0.0,NaN,NaN,NaN,1
28631,21863.0,Fogg,Jeremiah,1000.00,36.0,Regiment under Lieut. Col. G. Reid New Hampsh...,NH,0.0,NaN,NaN,NaN,1
28632,21864.0,Fogg,Jeremiah,750.00,36.0,Regiment under Lieut. Col. G. Reid New Hampsh...,NH,0.0,NaN,NaN,NaN,1
28633,21865.0,Fogg,Jeremiah,588.69,36.0,Regiment under Lieut. Col. G. Reid New Hampsh...,NH,0.0,NaN,NaN,NaN,1
28634,21866.0,Fogg,Jeremiah,50.00,36.0,Regiment under Lieut. Col. G. Reid New Hampsh...,NH,0.0,NaN,NaN,NaN,1


In [484]:
liquid1_JF

,state,Register Page,JPEG number,Number,Letter,Date of the Certificate: Month,Date of the Certificate: Day,Date of the Certificate: Year,First name,Last name,...,First name 3,Last name 3,8th,Date of the Certificate: Date,Line Strike Thorugh?,10th,Exchange,Amount in Specie_1,Amount in Specie_2,exact


In [485]:
print(
    f"loan: {loan1_JF['Face Value'].sum()} \n" + 
    f"marine: {marine1_JF['Dollars'].sum()} \n" + 
    f"pierce: {pierce1_JF['Value'].sum()} \n" + 
    f"liquid: {liquid1_JF['Dollars'].sum()}"
)

loan: 1900 
marine: 0.0 
pierce: 2859.39 
liquid: 0.0


### 'Joseph Mills'

In [486]:
f_names, l_name = ['Joseph'], 'Mills'

loan1_JM = get_result1(loan, f_names, l_name)
marine1_JM = get_result1(marine, f_names, l_name)
pierce1_JM = get_result1(pierce, f_names, l_name)
liquid1_JM = get_result1(df_liquid, f_names, l_name)

In [487]:
loan1_JM

,State,Year,Month,Day,Title 1,First name,Last name,Title 2,First Name 2,Last Name 2,Title 3,First Name 3,Last Name 3,Face Value,Specie Value,notes,original text,match eligible,exact
47008,6,1778,11,26.0,NaN,Joseph,Mills,NaN,NaN,NaN,NaN,NaN,NaN,500,80.0625,NaN,NaN,NaN,1
47009,6,1778,11,26.0,NaN,Joseph,Mills,NaN,NaN,NaN,NaN,NaN,NaN,500,80.0625,NaN,NaN,NaN,1
47010,6,1778,11,26.0,NaN,Joseph,Mills,NaN,NaN,NaN,NaN,NaN,NaN,500,80.0625,NaN,NaN,NaN,1
47011,6,1778,11,26.0,NaN,Joseph,Mills,NaN,NaN,NaN,NaN,NaN,NaN,500,80.0625,NaN,NaN,NaN,1


In [488]:
marine1_JM

,Page,JPEG number,Number,Letter,Date of the Certificate: Month,Date of the Certificate: Day,Date of the Certificate: Year,First name,Last name,Title,...,Total Dollars_1,Total Dollars_2,Line Strike Thorugh: Yes?,Line Strike Thorugh: Note,Notes,original text,cleaning notes,first name 2,last name 2,exact


In [489]:
pierce1_JM = pierce1_JM.query("exact==1")

In [490]:
liquid1_JM

,state,Register Page,JPEG number,Number,Letter,Date of the Certificate: Month,Date of the Certificate: Day,Date of the Certificate: Year,First name,Last name,...,First name 3,Last name 3,8th,Date of the Certificate: Date,Line Strike Thorugh?,10th,Exchange,Amount in Specie_1,Amount in Specie_2,exact
123,NH,NaN,94.0,124.0,E,7.0,28.0,1784.0,Joseph,Mills,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [491]:
print(
    f"loan: {loan1_JM['Face Value'].sum()} \n" + 
    f"marine: {marine1_JM['Dollars'].sum()} \n" + 
    f"pierce: {pierce1_JM['Value'].sum()} \n" + 
    f"liquid: {liquid1_JM['Dollars'].sum()}"
)

loan: 2000 
marine: 0.0 
pierce: 2396.4599999999996 
liquid: 13.0


### 'John Taylor Gilman'

In [492]:
f_names, l_name = ['John', 'John Taylor'], 'Gilman'
# Nicholas Gilman is his brother , 'Nicholas'

loan1_JTG = get_result1(loan, f_names, l_name)
marine1_JTG = get_result1(marine, f_names, l_name)
pierce1_JTG = get_result1(pierce, f_names, l_name)
liquid1_JTG = get_result1(df_liquid, f_names, l_name)

In [493]:
loan1_JTG

,State,Year,Month,Day,Title 1,First name,Last name,Title 2,First Name 2,Last Name 2,Title 3,First Name 3,Last Name 3,Face Value,Specie Value,notes,original text,match eligible,exact
1437,1,1779,2,27.0,NaN,John,Gilman,NaN,NaN,NaN,NaN,NaN,NaN,500,50.50000,NaN,NaN,NaN,1
13905,2,1779,5,29.0,NaN,John,Gillman,NaN,NaN,NaN,NaN,NaN,NaN,200,15.04722,NaN,NaN,NaN,0
1912,1,1780,2,16.0,NaN,John Taylor,Gilman,NaN,NaN,NaN,NaN,NaN,NaN,2000,56.61111,NaN,NaN,NaN,1
1951,1,1780,4,1.0,NaN,John Taylor,Gilman,NaN,NaN,NaN,NaN,NaN,NaN,10000,250.00000,NaN,NaN,NaN,1


In [494]:
marine1_JTG

,Page,JPEG number,Number,Letter,Date of the Certificate: Month,Date of the Certificate: Day,Date of the Certificate: Year,First name,Last name,Title,...,Total Dollars_1,Total Dollars_2,Line Strike Thorugh: Yes?,Line Strike Thorugh: Note,Notes,original text,cleaning notes,first name 2,last name 2,exact


In [495]:
pierce1_JTG

,CN,Last name,First name,Value,Group,To Whom Issued,State,Officer,Last 2,First 2,original text,exact
32393,34438.0,Gilman,John,41.6,51.0,Dragoons Col. E. Sheldon Connecticut paid t...,CT,0.0,NaN,NaN,NaN,1


In [496]:
liquid1_JTG

,state,Register Page,JPEG number,Number,Letter,Date of the Certificate: Month,Date of the Certificate: Day,Date of the Certificate: Year,First name,Last name,...,First name 3,Last name 3,8th,Date of the Certificate: Date,Line Strike Thorugh?,10th,Exchange,Amount in Specie_1,Amount in Specie_2,exact


In [497]:
print(
    f"loan: {loan1_JTG['Face Value'].sum()} \n" + 
    f"marine: {marine1_JTG['Dollars'].sum()} \n" + 
    f"pierce: {pierce1_JTG['Value'].sum()} \n" + 
    f"liquid: {liquid1_JTG['Dollars'].sum()}"
)

loan: 12700 
marine: 0.0 
pierce: 41.6 
liquid: 0.0


### 'Joshua Brackett'

In [498]:
f_names, l_name = ['Joshua'], 'Brackett'

loan1_JB = get_result1(loan, f_names, l_name)
marine1_JB = get_result1(marine, f_names, l_name)
pierce1_JB = get_result1(pierce, f_names, l_name)
liquid1_JB = get_result1(df_liquid, f_names, l_name)

In [499]:
loan1_JB

,State,Year,Month,Day,Title 1,First name,Last name,Title 2,First Name 2,Last Name 2,Title 3,First Name 3,Last Name 3,Face Value,Specie Value,notes,original text,match eligible,exact
612,1,1777,5,29.0,Esq,Joshua,Brackett,NaN,NaN,NaN,NaN,NaN,NaN,300,300.00000,NaN,NaN,NaN,1
613,1,1777,5,29.0,Esq,Joshua,Brackett,NaN,NaN,NaN,NaN,NaN,NaN,300,300.00000,NaN,NaN,NaN,1
614,1,1777,5,29.0,Esq,Joshua,Brackett,NaN,NaN,NaN,NaN,NaN,NaN,300,300.00000,NaN,NaN,NaN,1
651,1,1777,8,20.0,Esq,Joshua,Brackett,NaN,NaN,NaN,NaN,NaN,NaN,300,300.00000,NaN,NaN,NaN,1
935,1,1779,4,7.0,Esq,Joshua,Brackett,NaN,NaN,NaN,NaN,NaN,NaN,300,26.64444,NaN,NaN,NaN,1
936,1,1779,4,7.0,Esq,Joshua,Brackett,NaN,NaN,NaN,NaN,NaN,NaN,300,26.64444,NaN,NaN,NaN,1
1075,1,1777,5,29.0,Esq,Joshua,Brackett,NaN,NaN,NaN,NaN,NaN,NaN,400,400.00000,NaN,NaN,NaN,1
1448,1,1779,4,7.0,Esq,Joshua,Brackett,NaN,NaN,NaN,NaN,NaN,NaN,500,44.42222,NaN,NaN,NaN,1


In [500]:
marine1_JB

,Page,JPEG number,Number,Letter,Date of the Certificate: Month,Date of the Certificate: Day,Date of the Certificate: Year,First name,Last name,Title,...,Total Dollars_1,Total Dollars_2,Line Strike Thorugh: Yes?,Line Strike Thorugh: Note,Notes,original text,cleaning notes,first name 2,last name 2,exact


In [501]:
pierce1_JB

,CN,Last name,First name,Value,Group,To Whom Issued,State,Officer,Last 2,First 2,original text,exact


In [502]:
liquid1_JB

,state,Register Page,JPEG number,Number,Letter,Date of the Certificate: Month,Date of the Certificate: Day,Date of the Certificate: Year,First name,Last name,...,First name 3,Last name 3,8th,Date of the Certificate: Date,Line Strike Thorugh?,10th,Exchange,Amount in Specie_1,Amount in Specie_2,exact
276,MA,NaN,60.0,277.0,H,10.0,17.0,1783.0,Joshua,Bracket,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,NH,NaN,89.0,2.0,L,8.0,27.0,1783.0,Joshua,Brackett,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,NH,NaN,89.0,3.0,G,8.0,27.0,1783.0,Joshua,Brackett,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [503]:
print(
    f"loan: {loan1_JB['Face Value'].sum()} \n" + 
    f"marine: {marine1_JB['Dollars'].sum()} \n" + 
    f"pierce: {pierce1_JB['Value'].sum()} \n" + 
    f"liquid: {liquid1_JB['Dollars'].sum()}"
)

loan: 2700 
marine: 0.0 
pierce: 0.0 
liquid: 45.0


### 'John Lane'

In [504]:
f_names, l_name = ['John'], 'Lane'

loan1_JL = get_result1(loan, f_names, l_name)
marine1_JL = get_result1(marine, f_names, l_name)
pierce1_JL = get_result1(pierce, f_names, l_name)
liquid1_JL = get_result1(df_liquid, f_names, l_name)

In [505]:
loan1_JL = loan1_JL.query("exact==1")

In [506]:
marine1_JL

,Page,JPEG number,Number,Letter,Date of the Certificate: Month,Date of the Certificate: Day,Date of the Certificate: Year,First name,Last name,Title,...,Total Dollars_1,Total Dollars_2,Line Strike Thorugh: Yes?,Line Strike Thorugh: Note,Notes,original text,cleaning notes,first name 2,last name 2,exact


In [507]:
pierce1_JL = pierce1_JL.query("exact==1")

In [508]:
liquid1_JL = liquid1_JL.query("exact==1")

In [509]:
print(
    f"loan: {loan1_JL['Face Value'].sum()} \n" + 
    f"marine: {marine1_JL['Dollars'].sum()} \n" + 
    f"pierce: {pierce1_JL['Value'].sum()} \n" + 
    f"liquid: {liquid1_JL['Dollars'].sum()}"
)

loan: 200 
marine: 0.0 
pierce: 512.81 
liquid: 1.0
